In [ ]:
!pip install fpdf2 openpyxl pyttsx3 --quiet

In [1]:
import os
import datetime
import pyttsx3 # type: ignore
from fpdf import FPDF
from openpyxl import Workbook

ModuleNotFoundError: No module named 'pyttsx3'

In [2]:
class Producto:
    def __init__(self, nombre, precio, id_producto):
        self.nombre = nombre
        self.precio = precio
        self.id_producto = id_producto

    def get_precio(self):
        return self.precio

    def __str__(self):
        return f"{self.nombre} - ${self.precio:.2f}"

In [3]:
lass Carrito:
    def __init__(self):
        self.productos = {}

    def agregar_producto(self, producto, cantidad=1):
        if producto.id_producto in self.productos:
            self.productos[producto.id_producto]['cantidad'] += cantidad
        else:
            self.productos[producto.id_producto] = {'producto': producto, 'cantidad': cantidad}

    def eliminar_producto(self, id_producto):
        if id_producto in self.productos:
            del self.productos[id_producto]

    def modificar_cantidad(self, id_producto, nueva_cantidad):
        if id_producto in self.productos:
            if nueva_cantidad <= 0:
                self.eliminar_producto(id_producto)
            else:
                self.productos[id_producto]['cantidad'] = nueva_cantidad

    def calcular_total(self, descuento_porcentaje=0):
        total = sum(p['producto'].precio * p['cantidad'] for p in self.productos.values())
        descuento = total * (descuento_porcentaje / 100)
        return total - descuento

    def __len__(self):
        return sum(p['cantidad'] for p in self.productos.values())

    def exportar_resumen(self, descuento=0):
        fecha = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        resumen = f"Resumen de compra - {fecha}"

        wb = Workbook()
        ws = wb.active
        ws.title = "Resumen Compra"
        ws.append(["ID", "Producto", "Precio", "Cantidad", "Subtotal"])
        for p in self.productos.values():
            prod = p['producto']
            subtotal = prod.precio * p['cantidad']
            ws.append([prod.id_producto, prod.nombre, prod.precio, p['cantidad'], subtotal])
        total = self.calcular_total(descuento)
        ws.append([])
        ws.append(["Total con descuento", total])
        wb.save("resumen_compra.xlsx")

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=resumen, ln=True, align='C')
        pdf.ln(10)
        for p in self.productos.values():
            prod = p['producto']
            subtotal = prod.precio * p['cantidad']
            pdf.cell(200, 10, txt=f"{prod} x {p['cantidad']} = ${subtotal:.2f}", ln=True)
        pdf.ln(10)
        pdf.cell(200, 10, txt=f"Total con {descuento}% de descuento: ${total:.2f}", ln=True)
        pdf.output("resumen_compra.pdf")

        engine = pyttsx3.init()
        resumen_audio = f"Resumen de la compra. Usted ha comprado {len(self)} productos. El total con descuento es de {total:.2f} dólares."
        engine.save_to_file(resumen_audio, "resumen_compra.mp3")
        engine.runAndWait()

SyntaxError: invalid syntax (3968064160.py, line 1)